<a href="https://colab.research.google.com/github/Samuel-jesusboy/LLM_Practices/blob/main/Movie_Recommendation_with_Gemma_and_MongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets pandas pymongo sentence_transformers
!pip install -U transformers
# Install below if using GPU
!pip install accelerate

In [5]:
# Load Dataset
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/MongoDB/embedded_movies
dataset = load_dataset("MongoDB/embedded_movies")

# Convert the dataset to a pandas dataframe
dataset_df = pd.DataFrame(dataset["train"])

dataset_df.head(5)

,genres,title,num_mflix_comments,runtime,plot,writers,rated,directors,poster,type,plot_embedding,imdb,metacritic,languages,fullplot,cast,countries,awards
0,[Action],The Perils of Pauline,0,199.0,Young Pauline is left a lot of money when her ...,"[Charles W. Goddard (screenplay), Basil Dickey...",None,"[Louis J. Gasnier, Donald MacKenzie]",https://m.media-amazon.com/images/M/MV5BMzgxOD...,movie,"[0.00072939653, -0.026834568, 0.013515796, -0....","{'id': 4465, 'rating': 7.6, 'votes': 744}",NaN,[English],Young Pauline is left a lot of money when her ...,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",[USA],"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
1,"[Comedy, Short, Action]",From Hand to Mouth,0,22.0,A penniless young man tries to save an heiress...,[H.M. Walker (titles)],TV-G,"[Alfred J. Goulding, Hal Roach]",https://m.media-amazon.com/images/M/MV5BNzE1OW...,movie,"[-0.022837115, -0.022941574, 0.014937485, -0.0...","{'id': 10146, 'rating': 7.0, 'votes': 639}",NaN,[English],As a penniless man worries about how he will m...,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",[USA],"{'nominations': 1, 'text': '1 nomination.', 'w..."
2,"[Action, Adventure, Drama]",Beau Geste,0,101.0,"Michael ""Beau"" Geste leaves England in disgrac...","[Herbert Brenon (adaptation), John Russell (ad...",None,[Herbert Brenon],None,movie,"[0.00023330493, -0.028511643, 0.014653289, -0....","{'id': 16634, 'rating': 6.9, 'votes': 222}",NaN,[English],"Michael ""Beau"" Geste leaves England in disgrac...","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",[USA],"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
3,"[Adventure, Action]",The Black Pirate,1,88.0,"Seeking revenge, an athletic young man joins t...","[Douglas Fairbanks (story), Jack Cunningham (a...",None,[Albert Parker],https://m.media-amazon.com/images/M/MV5BMzU0ND...,movie,"[-0.005927917, -0.033394486, 0.0015323418, -0....","{'id': 16654, 'rating': 7.2, 'votes': 1146}",NaN,None,A nobleman vows to avenge the death of his fat...,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",[USA],"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
4,"[Action, Comedy, Romance]",For Heaven's Sake,0,58.0,An irresponsible young millionaire changes his...,"[Ted Wilde (story), John Grey (story), Clyde B...",PASSED,[Sam Taylor],https://m.media-amazon.com/images/M/MV5BMTcxMT...,movie,"[-0.0059373598, -0.026604708, -0.0070914757, -...","{'id': 16895, 'rating': 7.6, 'votes': 918}",NaN,[English],"The Uptown Boy, J. Harold Manners (Lloyd) is a...","[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",[USA],"{'nominations': 1, 'text': '1 nomination.', 'w..."


In [6]:
dataset_df['genres'].value_counts()

genres
[Action, Crime, Drama]          169
[Action, Adventure, Comedy]     112
[Action, Comedy, Crime]         100
[Action, Adventure, Drama]       90
[Action, Crime, Thriller]        56
                               ... 
[Action, Drama, Music]            1
[Sci-Fi, Action, Comedy]          1
[Action, Sci-Fi, Horror]          1
[Drama, Thriller, Action]         1
[Documentary, Action, Drama]      1
Name: count, Length: 164, dtype: int64

In [7]:
import pprint
pprint.pprint(dataset_df.iloc[0]['fullplot'])

('Young Pauline is left a lot of money when her wealthy uncle dies. However, '
 "her uncle's secretary has been named as her guardian until she marries, at "
 'which time she will officially take possession of her inheritance. '
 'Meanwhile, her "guardian" and his confederates constantly come up with '
 'schemes to get rid of Pauline so that he can get his hands on the money '
 'himself.')


In [8]:
# Data Preparation

# Remove data point where plot coloumn is missing
dataset_df = dataset_df.dropna(subset=["fullplot"])
print("\nNumber of missing values in each column after removal:")
print(dataset_df.isnull().sum())

# Remove the plot_embedding from each data point in the dataset as we are going to create new embeddings with an open source embedding model from Hugging Face
dataset_df = dataset_df.drop(columns=["plot_embedding"])
dataset_df.head(5)


Number of missing values in each column after removal:
genres                  0
title                   0
num_mflix_comments      0
runtime                14
plot                    0
writers                13
rated                 279
directors              12
poster                 78
type                    0
plot_embedding          1
imdb                    0
metacritic            893
languages               1
fullplot                0
cast                    1
countries               0
awards                  0
dtype: int64


,genres,title,num_mflix_comments,runtime,plot,writers,rated,directors,poster,type,imdb,metacritic,languages,fullplot,cast,countries,awards
0,[Action],The Perils of Pauline,0,199.0,Young Pauline is left a lot of money when her ...,"[Charles W. Goddard (screenplay), Basil Dickey...",None,"[Louis J. Gasnier, Donald MacKenzie]",https://m.media-amazon.com/images/M/MV5BMzgxOD...,movie,"{'id': 4465, 'rating': 7.6, 'votes': 744}",NaN,[English],Young Pauline is left a lot of money when her ...,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",[USA],"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
1,"[Comedy, Short, Action]",From Hand to Mouth,0,22.0,A penniless young man tries to save an heiress...,[H.M. Walker (titles)],TV-G,"[Alfred J. Goulding, Hal Roach]",https://m.media-amazon.com/images/M/MV5BNzE1OW...,movie,"{'id': 10146, 'rating': 7.0, 'votes': 639}",NaN,[English],As a penniless man worries about how he will m...,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",[USA],"{'nominations': 1, 'text': '1 nomination.', 'w..."
2,"[Action, Adventure, Drama]",Beau Geste,0,101.0,"Michael ""Beau"" Geste leaves England in disgrac...","[Herbert Brenon (adaptation), John Russell (ad...",None,[Herbert Brenon],None,movie,"{'id': 16634, 'rating': 6.9, 'votes': 222}",NaN,[English],"Michael ""Beau"" Geste leaves England in disgrac...","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",[USA],"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
3,"[Adventure, Action]",The Black Pirate,1,88.0,"Seeking revenge, an athletic young man joins t...","[Douglas Fairbanks (story), Jack Cunningham (a...",None,[Albert Parker],https://m.media-amazon.com/images/M/MV5BMzU0ND...,movie,"{'id': 16654, 'rating': 7.2, 'votes': 1146}",NaN,None,A nobleman vows to avenge the death of his fat...,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",[USA],"{'nominations': 0, 'text': '1 win.', 'wins': 1}"
4,"[Action, Comedy, Romance]",For Heaven's Sake,0,58.0,An irresponsible young millionaire changes his...,"[Ted Wilde (story), John Grey (story), Clyde B...",PASSED,[Sam Taylor],https://m.media-amazon.com/images/M/MV5BMTcxMT...,movie,"{'id': 16895, 'rating': 7.6, 'votes': 918}",NaN,[English],"The Uptown Boy, J. Harold Manners (Lloyd) is a...","[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",[USA],"{'nominations': 1, 'text': '1 nomination.', 'w..."


In [9]:
# Importing the necessary library
from sentence_transformers import SentenceTransformer

# Initializing the pre-trained model
# You can find more details about this model at: https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")

# Function to generate embeddings for text
def get_embedding(text: str) -> list[float]:
    # Check if the text is empty or whitespace
    if not text.strip():
        # Print a message if the text is empty
        print("Attempted to get embedding for empty text.")
        # Return an empty list
        return []

    # Generate the embedding for the input text
    embedding = embedding_model.encode(text)

    # Convert the embedding to a list of floating-point numbers
    return embedding.tolist()

# Applying the get_embedding function to each element in the "fullplot" column of the dataset
# Storing the embeddings in a new column named "embedding"
dataset_df["embedding"] = dataset_df["fullplot"].apply(get_embedding)

# Displaying the first few rows of the dataset with the newly added "embedding" column
dataset_df.head()

,genres,title,num_mflix_comments,runtime,plot,writers,rated,directors,poster,type,imdb,metacritic,languages,fullplot,cast,countries,awards,embedding
0,[Action],The Perils of Pauline,0,199.0,Young Pauline is left a lot of money when her ...,"[Charles W. Goddard (screenplay), Basil Dickey...",None,"[Louis J. Gasnier, Donald MacKenzie]",https://m.media-amazon.com/images/M/MV5BMzgxOD...,movie,"{'id': 4465, 'rating': 7.6, 'votes': 744}",NaN,[English],Young Pauline is left a lot of money when her ...,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",[USA],"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[-0.009285839274525642, -0.005062091629952192,..."
1,"[Comedy, Short, Action]",From Hand to Mouth,0,22.0,A penniless young man tries to save an heiress...,[H.M. Walker (titles)],TV-G,"[Alfred J. Goulding, Hal Roach]",https://m.media-amazon.com/images/M/MV5BNzE1OW...,movie,"{'id': 10146, 'rating': 7.0, 'votes': 639}",NaN,[English],As a penniless man worries about how he will m...,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",[USA],"{'nominations': 1, 'text': '1 nomination.', 'w...","[-0.002439370146021247, 0.023095937445759773, ..."
2,"[Action, Adventure, Drama]",Beau Geste,0,101.0,"Michael ""Beau"" Geste leaves England in disgrac...","[Herbert Brenon (adaptation), John Russell (ad...",None,[Herbert Brenon],None,movie,"{'id': 16634, 'rating': 6.9, 'votes': 222}",NaN,[English],"Michael ""Beau"" Geste leaves England in disgrac...","[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",[USA],"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[0.012204294092953205, -0.011455751955509186, ..."
3,"[Adventure, Action]",The Black Pirate,1,88.0,"Seeking revenge, an athletic young man joins t...","[Douglas Fairbanks (story), Jack Cunningham (a...",None,[Albert Parker],https://m.media-amazon.com/images/M/MV5BMzU0ND...,movie,"{'id': 16654, 'rating': 7.2, 'votes': 1146}",NaN,None,A nobleman vows to avenge the death of his fat...,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",[USA],"{'nominations': 0, 'text': '1 win.', 'wins': 1}","[0.004541350528597832, -0.0006100559257902205,..."
4,"[Action, Comedy, Romance]",For Heaven's Sake,0,58.0,An irresponsible young millionaire changes his...,"[Ted Wilde (story), John Grey (story), Clyde B...",PASSED,[Sam Taylor],https://m.media-amazon.com/images/M/MV5BMTcxMT...,movie,"{'id': 16895, 'rating': 7.6, 'votes': 918}",NaN,[English],"The Uptown Boy, J. Harold Manners (Lloyd) is a...","[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",[USA],"{'nominations': 1, 'text': '1 nomination.', 'w...","[-0.002225600415840745, 0.011567802168428898, ..."


In [10]:
# Importing the pymongo library for interacting with MongoDB
import pymongo
# Importing the userdata module from google.colab to retrieve environment variables
from google.colab import userdata

# Function to establish connection to MongoDB
def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

mongo_uri = userdata.get("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
# Creating a database named 'movies_recommend'
db = mongo_client['movies_recommend']
# Creating a collection named 'movie_collection_2'
collection = db['record']

Connection to MongoDB successful


In [11]:
collection

Collection(Database(MongoClient(host=['ac-jxdry4c-shard-00-01.dq7k0ai.mongodb.net:27017', 'ac-jxdry4c-shard-00-02.dq7k0ai.mongodb.net:27017', 'ac-jxdry4c-shard-00-00.dq7k0ai.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='Cluster0', authsource='admin', replicaset='atlas-5loth6-shard-0', tls=True), 'movies_recommend'), 'record')

In [12]:
# Delete any existing records in the collection
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff00000000000000ed'), 'opTime': {'ts': Timestamp(1712861771, 36), 't': 237}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1712861771, 36), 'signature': {'hash': b'\xd4\xd8\x07\x12Y1\xe3\x02v\xf3\x9c8$\xe62\xa5\x85\xc76%', 'keyId': 7315130655791120385}}, 'operationTime': Timestamp(1712861771, 36)}, acknowledged=True)

In [13]:
documents = dataset_df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


In [14]:

def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    vector_search_stage = {
        "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": 150,  # Number of candidate matches to consider
            "limit": 4  # Return top 4 matches
        }
    }

    unset_stage = {
        "$unset": "embedding"  # Exclude the 'embedding' field from the results
    }

    project_stage = {
        "$project": {
            "_id": 0,  # Exclude the _id field
            "fullplot": 1,  # Include the plot field
            "title": 1,  # Include the title field
            "genres": 1, # Include the genres field
            "score": {
                "$meta": "vectorSearchScore"  # Include the search score
            }
        }
    }

    pipeline = [vector_search_stage, unset_stage, project_stage]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

In [15]:
def get_search_result(query, collection):

    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('fullplot', 'N/A')}\n"

    return search_result

In [39]:
# Conduct query with retrival of sources
query = "give me the top 15 jackie chan movies and their ratings?"
source_information = get_search_result(query, collection)
combined_information = f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}."

print(combined_information)

Query: give me the top 15 jackie chan movies and their ratings?
Continue to answer the query by using the Search Results:
Title: Miracles - Mr. Canton and Lady Rose, Plot: Jackie Chan's Hong Kong variation of Frank Capra's "A Pocketful of Miracles" set in the 1930s. Jackie plays a country boy who rescues a gang boss. Jackie becomes the head of a gang through the purchase of some lucky roses from an old lady. Jackie and a singer at the gang's nightclub try to do a good deed for the old rose-seller when her daughter comes to visit, all this while battling a rival gang.
Title: The Myth, Plot: Martial arts legend Jackie Chan stars as Jack, a world-renowned archaeologist who has begun having mysterious dreams of a past life as a warrior in ancient China. When a fellow scientist enlists his help locating the mausoleum of China's first emperor, the past collides violently with the present as Jack discovers his amazing visions are based in fact. Assisted by the spirit of a noble princess...
Ti

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# CPU Enabled uncomment below 👇🏽
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
# GPU Enabled use below 👇🏽
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [40]:
# Moving tensors to GPU
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=500)
print(tokenizer.decode(response[0]))

<bos>Query: give me the top 15 jackie chan movies and their ratings?
Continue to answer the query by using the Search Results:
Title: Miracles - Mr. Canton and Lady Rose, Plot: Jackie Chan's Hong Kong variation of Frank Capra's "A Pocketful of Miracles" set in the 1930s. Jackie plays a country boy who rescues a gang boss. Jackie becomes the head of a gang through the purchase of some lucky roses from an old lady. Jackie and a singer at the gang's nightclub try to do a good deed for the old rose-seller when her daughter comes to visit, all this while battling a rival gang.
Title: The Myth, Plot: Martial arts legend Jackie Chan stars as Jack, a world-renowned archaeologist who has begun having mysterious dreams of a past life as a warrior in ancient China. When a fellow scientist enlists his help locating the mausoleum of China's first emperor, the past collides violently with the present as Jack discovers his amazing visions are based in fact. Assisted by the spirit of a noble princess.

In [37]:
dataset_df[dataset_df['title']=='Miracles']

,genres,title,num_mflix_comments,runtime,plot,writers,rated,directors,poster,type,imdb,metacritic,languages,fullplot,cast,countries,awards,embedding
